In [1]:
# ==========================================
# 1. SETUP & OPTIMIZATION
# ==========================================
# Force specific protobuf to prevent TensorFlow crash
!pip install protobuf==3.20.3 --quiet

import tensorflow as tf
from tensorflow.keras import layers, models, mixed_precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
import matplotlib.pyplot as plt

# Enable Mixed Precision (Doubles speed on P100 GPU)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

print("🚀 Hardware Status:")
print(f"Num GPUs Available: {len(tf.config.list_physical_devices('GPU'))}")
print(f"GPU Name: {tf.test.gpu_device_name()}")

2025-12-11 04:31:36.199753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765427496.378932     291 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765427496.433641     291 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🚀 Hardware Status:
Num GPUs Available: 1
GPU Name: /device:GPU:0


I0000 00:00:1765427506.133210     291 gpu_device.cc:2022] Created device /device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [2]:
# ==========================================
# 2. DATASET PATH CONFIGURATION
# ==========================================
import os

# Direct path assignment
dataset_path = '/kaggle/input/plant-disease-detection/Plant_Disease_Dataset'

if os.path.exists(dataset_path):
    print(f"✅ Dataset Root Found: {dataset_path}")
    print(f"   Contents: {len(os.listdir(dataset_path))} items")
else:
    print(f"❌ CRITICAL ERROR: Path not found at {dataset_path}")

✅ Dataset Root Found: /kaggle/input/plant-disease-detection/Plant_Disease_Dataset
   Contents: 34 items


In [3]:
# ==========================================
# 3. DATA PIPELINE (Safe Streaming Mode)
# ==========================================
# Reduced to 32 to prevent VRAM spikes
BATCH_SIZE = 32 
IMG_SIZE = (224, 224)

print("⏳ Loading Training Data...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

print("⏳ Loading Validation Data...")
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"\n🏷️ Classes: {class_names}")

# OPTIMIZATION: Streaming Only (No RAM Caching)
# We removed .cache() to prevent the 17GB RAM crash.
# We keep prefetch() to keep the GPU busy.
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

print("✅ Data Pipeline Ready (Streaming Mode)")

⏳ Loading Training Data...
Found 32640 files belonging to 34 classes.
Using 26112 files for training.


I0000 00:00:1765427616.848750     291 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


⏳ Loading Validation Data...
Found 32640 files belonging to 34 classes.
Using 6528 files for validation.

🏷️ Classes: ['corn_common_rust', 'corn_gray_leaf_spot', 'corn_healthy', 'corn_northern_leaf_blight', 'pepper_bell_bacterial_spot', 'pepper_bell_healthy', 'potato_early_blight', 'potato_healthy', 'potato_late_blight', 'rice_bacterial_leaf_blight', 'rice_brown_spot', 'rice_healthy', 'rice_leaf_blast', 'rice_leaf_smut', 'rice_neck_blast', 'sugarcane_bacterial_blight', 'sugarcane_healthy', 'sugarcane_red_rot', 'sugarcane_white_leaf', 'tomato_bacterial_spot', 'tomato_early_blight', 'tomato_healthy', 'tomato_late_blight', 'tomato_leaf_mold', 'tomato_mosaic_virus', 'tomato_septoria_leaf_spot', 'tomato_spider_mites_two_spotted_spider_mite', 'tomato_target_spot', 'tomato_yellowLeaf_curl_virus', 'wheat_brown_rust', 'wheat_healthy', 'wheat_septoria', 'wheat_stripe_rust', 'wheat_yellow_rust']
✅ Data Pipeline Ready (Streaming Mode)


In [4]:
# ==========================================
# 4. MODEL ARCHITECTURE
# ==========================================
from tensorflow.keras.applications import MobileNetV2

# Data Augmentation (Runs on GPU)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
])

# Base Model (Pre-trained)
base_model = MobileNetV2(input_shape=(224, 224, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False 

# Full Architecture
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x) 

# Output Layer (Must be float32 for stability)
outputs = layers.Dense(len(class_names), activation='softmax', dtype='float32')(x)

model = models.Model(inputs, outputs)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("✅ Model Built.")

✅ Model Built.


In [5]:
# ==========================================
# 5. TRAIN & EXPORT
# ==========================================
import json

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('/kaggle/working/best_disease_model.keras', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
]

print("🚀 Starting Training (Sit back, this takes ~20 mins)...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks
)



🚀 Starting Training (Sit back, this takes ~20 mins)...
Epoch 1/15


I0000 00:00:1765427765.796354     353 cuda_dnn.cc:529] Loaded cuDNN version 90300


816/816 ━━━━━━━━━━━━━━━━━━━━ 179s 84ms/step - accuracy: 0.6020 - loss: 1.3471 - val_accuracy: 0.8390 - val_loss: 0.4987 - learning_rate: 0.0010
Epoch 2/15
816/816 ━━━━━━━━━━━━━━━━━━━━ 132s 74ms/step - accuracy: 0.8251 - loss: 0.5094 - val_accuracy: 0.8560 - val_loss: 0.4261 - learning_rate: 0.0010
Epoch 3/15
816/816 ━━━━━━━━━━━━━━━━━━━━ 128s 73ms/step - accuracy: 0.8374 - loss: 0.4642 - val_accuracy: 0.8439 - val_loss: 0.4316 - learning_rate: 0.0010
Epoch 4/15
816/816 ━━━━━━━━━━━━━━━━━━━━ 131s 74ms/step - accuracy: 0.8467 - loss: 0.4402 - val_accuracy: 0.8681 - val_loss: 0.3893 - learning_rate: 0.0010
Epoch 5/15
816/816 ━━━━━━━━━━━━━━━━━━━━ 129s 73ms/step - accuracy: 0.8586 - loss: 0.4070 - val_accuracy: 0.8820 - val_loss: 0.3500 - learning_rate: 0.0010
Epoch 6/15
816/816 ━━━━━━━━━━━━━━━━━━━━ 128s 72ms/step - accuracy: 0.8621 - loss: 0.3971 - val_accuracy: 0.8797 - val_loss: 0.3436 - learning_rate: 0.0010
Epoch 7/15
816/816 ━━━━━━━━━━━━━━━━━━━━ 126s 72ms/step - accuracy: 0.8604 - loss:

In [6]:
# SAVE FINAL FILES
print("\n💾 Saving Artifacts...")
model.save('/kaggle/working/final_disease_model.h5')

with open('/kaggle/working/class_indices.json', 'w') as f:
    json.dump(class_names, f)

print("✅ DONE! Go to the 'Output' tab (right sidebar) to download 'final_disease_model.h5' and 'class_indices.json'.")


💾 Saving Artifacts...
✅ DONE! Go to the 'Output' tab (right sidebar) to download 'final_disease_model.h5' and 'class_indices.json'.
